# Lab Week 7: Nonlinear Least Squares and Trust-Region Methods

_Tutors: Andreas Grothey, Josh Fogg_

7 March 2025

This lab session is concerned with the concepts of the lectures in weeks 5 and 6: Nonlinear Least Squares and Trust Region Methods. There is again an Assessed Task at the end. First download the python modules required for today's session from Learn (`NLOLab4.zip`) and unpack it.

## 1. Nonlinear Least Squares + Gauss-Newton

Assume the following setup that we wish to solve with a nonlinear least squares approach: We have measurements of radioactivity levels that are generated by two different decaying components(different amounts and different half lives) plus an unknown degree of background radiation. The amount of radiation at time $t$ can thus be modelled as

$$
    \phi(t; r_0, c_1, c_2, \lambda_1, \lambda_2) = r_0 + c_1 e^{-\lambda_1 t} + c_2 e^{-\lambda_2 t}
$$

we would like to find the best fit of parameters $r_0$, $c_1$, $c_2$, $\lambda_1$, $\lambda_2$ to given measurement data $(t_j, \phi_j)_{j=1,\ldots,J}$. Some example data is plotted below.

In [ ]:
from doubleexpdecay import dat
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(dat[:,0], dat[:,1], s=10)
ax.set_title('example data (r0 = 3.5, c1 = 2, c2 = 4, l1 = 2, l2 = 0.5)')
ax.set_xlabel('time (min)')
ax.set_ylabel('radiation level')

For this we can use the least squares model
$$
    \min_x f(x) = \sum_{j=1}^J {(\phi(t_j; x) - \phi_j)}^2,\quad \text{where} x = (r_0, c_1, c_2, \lambda_1, \lambda_2).
$$
We have the derivatives
$$
    \nabla\phi(x) = \begin{pmatrix}
        1 \\
        e^{-\lambda_1 t} \\
        e^{-\lambda_2 t} \\
        -tc_1 e^{-\lambda_1 t} \\
        -tc_2 e^{-\lambda_2 t}
    \end{pmatrix},\quad

    \nabla^2\phi(x) = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & -te^{-\lambda_1 t} & 0 \\
        0 & 0 & 0 & 0 & -te^{-\lambda_2 t} \\
        0 & -te^{-\lambda_1 t} & 0 & t^2 c_1 e^{-\lambda_1 t} & 0 \\
        0 & 0 & -te^{-\lambda_2 t} & 0 & -t^2 c_2 e^{-\lambda_2 t}
    \end{pmatrix}
$$

This nonlinear least squares model is implemented in the python files `doubleexpdecay.py` (which gives the function $\phi$ and its derivatives plus the sample data) and `nls.py` (which defines the NLS objective $f(x)$ is the same format as out other test functions in the previous lab sessions).

### (a)

_empty_

### (b)

This is a normal unconstrained optimization problem, so before we try specialised methods it will be worthwhile to investigate how our standard algorithms perform. Lets start by trying Newtons methods (at the moment standard Newton, not the Gauss-Newton approximation). For this run the python script `CallLineSearch.py` from this weeks download. You can inspect the file, as in previous lab sessions it sets up the function to be minimized (here `nls.py`) and calls the generic line search method `GenericLineSearchMethod.py`. By default (as in the download) this uses Newton's method with full steps ($\alpha = 1$).

You will find that the method stops with an error message: The Newton direction is not a descent direction. Helpfully the line search method also reports the eigenvalues of the Hessian: this is indeed an indefinite matrix (if you allow Newton to continue without interfering it will converge to a saddle point).

### (c)

As a first attempt to overcome this we can use inertia correction (adding $\nu I$ to the Hessian, where $\nu > |\lambda_{\min}(\nabla^2f(x))|$. This can be enabled by changing the switch do Newton `InertiaCorr = True` at the top (parameter section) of `GenericLineSearchMethod.py`. (Note that this simply calculates eigenvalues of the Hessian and then uses $\nu = |\lambda_{\min}| + 10^{−4}$ for the correction).

Unfortunately this still results in an error: this time an overflow error (trying to evaluate the `exp()` function at a too large argument), due to Newton's method taking very large steps.

### (d)

The obvious fix would seem to be to not accept full steps ($\alpha = 1$) of the Newton direction but to do a line search. Switch to Armijo line search and see what happens.

### (e)

In summary plain Newton, does seem to have problems that are not easily solved (ideally we would impose bound constraints on the variables, that prevent values getting too large, but then we have to think how to incorporate them). Rather than doing this, lets try our other methods.

You may want to check that Conjugate Gradients with Armijo line search also runs into the same problem as Newtons methods (though it does work with exact line-searches – where is requires about 790 function evaluations).

### (f)

The inertia correction in Newton is a quite blunt instrument to enforce a descent direction. A more subtle approach to approximates the Hessian with a positive definite matrix (and thus guaranteeing descent directions) are Quasi-Newton methods. So, try BFGS by changing the search direction in `GenericLineSearchMethod.py` to `QN` and keeping the line search as `Armijo`. (Also reset the Newton inertia correction to `False`). Take a note of the final function value `f=...` that the algorithm converges to.

This is now much more successful and again shows the power of the QN methods.

### (g)

Now that we have been able to solve the problem we would like to plot the resulting fit. For this have a look at the python script `PlotNLSfit.py`. This loads the data from `doubleexpdecay` and plots it (to produce the plot at the top of this sheet). It also includes code to plot the fitted function if the values of $x = (r_0, c_1, c_2, \lambda_1, \lambda_2$) are provided (you will have to copy them by hand). Try this.

## 2. Gauss-Newton

### (a)

The Gauss-Newton method is designed to be a specialised method for the NLS setup. Further we have seen that the GN Hessian approximation is guaranteed to be at least positive semi-definite. Let's see if this can overcome the earlier problems with Newtons method: Change the setup to again use Newtons method (`direction = 'Newton'`, `linesearch = 'Armijo'`) but change the selector in `nls.py` to use `GN = True` (which result in returning the GN approximation to the Hessian, rather than the real Hessian). Try this.

### (b)

Unfortunately we are running into a similar problem as with Newton, as the algorithm stops on the first iteration with a "Singular Matrix" error. Remember that the Gauss-Newton approximation is only guaranteed to be positive semi-definite, not positive definite, so this can happen. Indeed we know from the lecture that this happens exactly when the Jacobian $J = (\nabla r_1, \ldots, \nabla r_m)^{\top}$ does not have full rank.

To overcome this, switch the inertia correction back on. Since now we do not have any strictly negative eigenvalues, the inertia correction will not make large changes to the Hessian. Indeed the Gauss-Newton algorithm with inertia correction is able to converge. Again, take note of the function value (=goodness of fit) that it converges to.

## 3. Trust-Region: The Levenberg-Marquardt Method

### (a)

We have seen that the line search methods struggle a lot with indefinite and semi-definite Hessian matrices and too large steps. Both of these are issues that Trust-Region methods are meant to be able to overcome. Let’s investigate how well a trust region method works for the above nonlinear least squares problem. For this use the python script `CallTrustRegion.py`. Again, before running it, inspect it. `CallTrustRegion.py` is almost the same as `CallLineSearch.py` except that it calls the generic trust region method `GenericTrustRegionMethod.py`. That in turn implements the Trust Region logic, there are parameters to set that fine-tune the trust region radius update and let you choose the subproblem solver (`'LM'` = Levenberg-Marquardt by default). It uses a second order Taylor model that obtains the derivatives at the current point from nls.py as for the line search methods.

You will still have use `GN=True` in `nls.py` from (2a/b), but try this with both options: Gauss-Newton and exact Newton.

Indeed it is **this method** (Levenberg-Marquardt as an L2-TR subproblem solver within a Gauss-Newton method to solve NLS problems) that is by most authors referred to as **the** Levenberg-Marquardt method.

### (b)

The breakdown of problem statistics for the TR method gives you separate values for the number of function evaluations and gradient evaluations. Further it reports the number of "solves in TR subproblem": that is the number of different $\nu$ values tried in the Levenberg-Marquardt subproblem (each of which requires to solve a linear system of equations for the step) until a $\nu$ with $\|d(\nu)\|^2 = \rho$ has been found. Note that the default settings for LM use a fairly loose criterion of $\big|\|d(\nu)\|^2 − \rho\big| < 0.05\rho$ to decide when to stop the $\nu$-loop.

### (c)

Finally try the use of an inexact TR solver by using the Cauchy-Point/Dog-Leg method. For this change the setting `subsolver = 'LM'` to `subsolver = 'DL'` in `GenericTrustRegionMethod.py`. Try with both the Gauss-Newton approximation and the true Hessian.

Note that the Dog-Leg method is only guaranteed to improve on the Cauchy-Point if the Hessian matrix is positive definite. Since we cannot guarantee this, the supplied Cauchy-Point/Dog-Leg solver `solveTRL2Cauchy.py` first evaluates the model at the stationary point of the model function $d^U = −B^{−1}g$. Only if that point is outside of the Trust Region but would improve on the Cauchy point is the second leg of the dog-leg path considered. Otherwise the subproblem solver just returns the Cauchy point.

Surprisingly the inexact Cauchy-Point/Dog-Leg method performs much better than the exact Levenberg-Marquardt method.

## 4. Assessed Task

> (a) Submit the plot for part 1(g): the data points together with the best fit.
>
> (b) Summarize which is the best line search method and the best TR method for this particular problem.
> 
> (c) Which is the best overall? How would you compare the work necessary for the best line search method and the best TR method?

Submission is on the NLO Learn pages (under Assessments) by **Friday 14 March 10am**.